In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.estimator import estimator
from tensorflow.python.estimator import model_fn

import numpy as np

/Users/teggsung/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [33]:
'''
TODO:
    input: two different dataset
        Call dataset operations
        densenet: L=200, k=12, dropout=0; ep=300, batch_size=64
        wideresnet: depth=28, width=10, dropout=0; ep=200, batch_size=128
        
    modules: input perturbation
             temperature scaling
             
    op: SGD with Nesterov momentum (0.9)
    lr: start=0.1, decay_rate=0.1, decay_time=[50,75]
    
    in-distribution datasets: CIFAR-10, CIFAR-100
    out-of-distribution datasets: TinyImageNet, LSUN, iSUN, Gaussian noise, Uniform noise'''

'\nTODO:\n    input: two different dataset\n        Call dataset operations\n        densenet: L=200, k=12, dropout=0; ep=300, batch_size=64\n        wideresnet: depth=28, width=10, dropout=0; ep=200, batch_size=128\n        \n    modules: input perturbation\n             temperature scaling\n             \n    op: SGD with Nesterov momentum (0.9)\n    lr: start=0.1, decay_rate=0.1, decay_time=[50,75]\n    \n    in-distribution datasets: CIFAR-10, CIFAR-100\n    out-of-distribution datasets: TinyImageNet, LSUN, iSUN, Gaussian noise, Uniform noise'

In [34]:
'''
TODO: Load actual datasets & preprocess to get ready'''

'\nTODO: Load actual datasets & preprocess to get ready'

In [17]:
## Temperature scaling
def temp_scaling(x, model, temperature):
    scaled = tf.nn.softmax_cross_entropy_with_logits_v2(x, model(x)/temperature)
    return scaled

In [19]:
## Image preprocessing
def preprocess(x, model, eps, temperature):
    perturbed = temp_scaling(x, model, temperature)
    prep_x = x - eps * np.sign(-perturbed)
    return prep_x

In [38]:
## Compare score
def check_in_out(x, y, model, thresh, temperature=1000, eps=0.0012):
    score = tf.nn.softmax_cross_entropy_with_logits_v2(preprocess(x, model, eps, temperature), y)
    if score >= thresh:
        print('in-distribution')
    else:
        print('out-of-distribution')

# Model code

In [39]:
def odin_classifier(dataset, model, mode, params):
#     labels = tf.cast(labels, tf.int32)
    
    ## Check available mode
    if mode not in [model_fn.ModeKeys.TRAIN, model_fn.ModeKeys.EVAL, model_fn.ModeKeys.PREDICT]:
        raise ValueError('Mode not recognized: {}'.format(mode))
        
#    assert isinstance() or isinstance()
    
    check_in(x=dataset1,
             y=dataset2,
             model=model,
             thresh=0.5)
    
    # Define training operations
    op =
    loss = 
    
    # Define accuracy, metrics
    metrics = 
    accuracy = 
    
    
    # Eval
    if mode == tf.estimator.ModeKeys.EVAL:
        spec = tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
        merged = tf.summary.merge_all()
        test_writer = tf.summary.FileWriter('test')
        return spec

    # Train: Alternative learning
    if mode == tf.estimator.ModeKeys.TRAIN:
        spec = tf.estimator.EstimatorSpec(
            mode, loss=loss, train_op=op)
        tf.summary.scalar('accuracy', accuracy[1])
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('train')
        return spec

    raise ValueError("Invalid estimator mode: reached the end of the function")

SyntaxError: invalid syntax (<ipython-input-39-ef702dcd7085>, line 16)

# Example code

In [2]:
## Metric
tf.metrics.auc(labels,  predictionns)
tf.metrics.accuracy(labels, predictions)
tf.metrics.precision(labels, predictions)
tf.metrics.recall(labels, predictions)
tf.metrics.average_precision_at_k(labels, predictions, k)

NameError: name 'labels' is not defined

In [19]:
def cnn_model_fn(features, labels, mode):
    input_layer = features['images']
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5,5],
        padding='same',
        activation=None)
    batch_norm1 = tf.layers.batch_normalization(conv1)
    relu1 = tf.nn.relu(batch_norm1)
    pool1 = tf.layers.max_pooling2d(inputs=relu1, pool_size=[2,2], strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding='same',
        activation=None)
    batch_norm2 = tf.layers.batch_normalization(conv2)
    relu2 = tf.nn.relu(batch_norm2)
    pool2 = tf.layers.max_pooling2d(inputs=relu2, pool_size=[2,2], strides=2)
    
    pool2_flat = tf.layers.flatten(pool2)
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, 
                                training={mode == tf.estimator.ModeKeys.TRAIN})
    
    logits = tf.layers.dense(inputs=dropout, units=params['num_classes'])
    
    predictions = {'classes': tf.argmax(input=logits, axis=1),
                   'probabilities': tf.nn.softmax(logits, name='softmax_tensor')}
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, 
            export_outputs={'predict': tf.estimator.export.PredictOutput(predictions)})
        return spec
        
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        spec = tf.estimator.EstimatorSpec(
            optimizer = tf.train.AdamOptimizer(),
            train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step()) )
        return spec
        
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes']),
        'aupr': tf.metrics.average_precision_at_k(labels, predictions, k=k)
    }
    spec = tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    return spec

In [14]:
'''
dataset1 = ...
dataset2 = ...
odin_clasifier(dataset1, dataset2)'''

'\ndataset1 = ...\ndataset2 = ...\nodin_clasifier(dataset1, dataset2)'

In [13]:
params = {
    'steps': 10000,
    'num_classes': 10,
    'model_dir': './ckpt/',
    'saved_dir': '/pb/',
}

In [21]:
training_config = tf.estimator.RunConfig(
    model_dir=params['model_dir'],
    save_summary_steps=20,
    save_checkpoints_steps=20)
classifier = tf.estimator.Estimator(
    config=training_config,
    model_fn=cnn_model_fn,
    model_dir=params['model_dir'])

INFO:tensorflow:Using config: {'_model_dir': './ckpt/', '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_steps': 20, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb20e46ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
tensors_to_log = {'probabilities': 'softmax_tensors'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [23]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': train_data},
    y=train_labels,
    batch_size=params['steps'],
    num_epochs=None,
    shuffle=True)
classifier.train(input_fn=train_input_fn, 
                 steps=params['steps'], 
                 hooks=[logging_hook])

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'image': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

NameError: name 'train_data' is not defined